In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from pathlib import Path
import pandas as pd

/tmp/ipykernel_395982/362687639.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [19]:
save_dir = '/home/jl277509/Documents/Ablation_2_models_full/rotation_translation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/rotation_translation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/rotation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/rotation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/translation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/7_translation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/cutin'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/10_cutin_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/cutin'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/11_cutout_translation_3'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/no_augm'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/6_no_augm'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/rotation_then_translation'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_full/rotation_then_translation'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/combinations'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_combinations/combinations'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/combinations_rotation_first'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_combinations/combinations_rotation_first'

save_dir = '/home/jl277509/Documents/Ablation_2_models_full/combinations_with_trim'
basedir='/neurospin/dico/jlaval/Output/ablation_2_models_combinations/combinations_with_trim'


#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimdepth_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/8_trimdepth_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/trimextremities_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/9_trimextremities_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutin_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/10_cutin_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/cutout_translation_3.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'

#save_dir = '/home/jl277509/Documents/Ablation_2_models/6_no_augm.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/11_cutout_translation_3'
#n_models = 2

#save_dir = '/home/jl277509/Documents/Ablation_2_models/rotation_translation.csv'
#basedir='/neurospin/dico/jlaval/Output/ablation_2_models/rotation_translation'
#n_models = 1


# Step 1 : recover all perf values (with cross val std)

In [20]:
sub_dirs = {'SOr': [f'{basedir}/SOr_left_UKB40/', ['troiani_custom_embeddings/Left_OFC/test_values.json'], 'cv_score', 'cv_std', 'splits_score'], # NB : error in pipeline stds -> std ... ## ONLY IN CUTOUT : rerun
            'FIP': [f'{basedir}/FIP_right_UKB40/', ['FIP_right_custom_embeddings/Right_FIP/test_values.json'], 'cv_score', 'cv_std', 'splits_score'],
            'SC-sylv': [f'{basedir}/SC-sylv_left_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_central_left_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_1': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', [f'hcp_isomap_custom_embeddings/Isomap_cingulate_right_dim{k}/test_values.json' for k in range(1,7)], 'cv_score', 'cv_std', 'splits_score'],
            'Cing_2': [f'{basedir}/LARGE_CINGULATE_right_UKB40/', ['ACC_custom_embeddings/Right_PCS/test_values.json'], 'cv_score', 'cv_std', 'splits_score']
            }

l = os.listdir(basedir)
root_names = [elem for elem in l if ('UKB40' not in elem) and '#' not in elem]

In [21]:
# store everything globally
l_model_glob = []
l_perf_glob = []
l_std_glob = []
l_splits_scores_glob = []
l_region_glob = []
l_task_glob = []


for region, (sub_dir, auc_files, score, std, split_score) in sub_dirs.items(): 
    for auc_file in auc_files:
        models = os.listdir(sub_dir)
        models = [elem for elem in models if 'config' not in elem and 'multirun' not in elem]
        models.sort()
        task = Path(auc_file).parts[-2]
        for name in root_names:
            #l_models = [elem for elem in models if elem.startswith(name)]
            l_models = [elem for elem in models if name in elem]
            l_perfs = []
            l_stds = []
            l_splits_scores = []
            for model in l_models:
                try:
                    with open(os.path.join(sub_dir, model, auc_file), 'r') as f:
                        perf = json.load(f)
                except Exception:
                    print(f'{os.path.join(sub_dir, model, auc_file)} not found') ## NB : can't continue if not all files found ...

                if isinstance(perf[score], float):
                    value = perf[score]
                else:
                    value = perf[score][0]
                l_perfs.append(value)
                if isinstance(perf[std], float):
                    value = perf[std]
                else:
                    value = perf[std][0]
                l_stds.append(value)
                value = perf[split_score] # is a list
                l_splits_scores.append(value)
                
            l_model_glob.append(l_models)
            l_perf_glob.append(l_perfs)
            l_std_glob.append(l_stds)
            l_splits_scores_glob.append(l_splits_scores)
            l_region_glob.append([region for _ in range(len(l_perfs))])
            l_task_glob.append([task for _ in range(len(l_perfs))])

In [22]:
# reformat lists and put inside DataFrame
def flatten_sum(matrix):
    return sum(matrix, [])
cols = ['model', 'region', 'task', 'score', 'std']
values = np.array([flatten_sum(l_model_glob), flatten_sum(l_region_glob), flatten_sum(l_task_glob), flatten_sum(l_perf_glob), flatten_sum(l_std_glob)]).T
df = pd.DataFrame(data=values, columns=cols)
df['splits_score']=flatten_sum(l_splits_scores_glob)

# Step 2 : reorganize the df with the parameters

In [41]:
## arbitrary parameter here
#df['No augm'] = [0 for k in range(len(df))]
parameters =[]

Rotation - Translation

In [8]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_0' in model:
        list_magnitude_trans.append(0)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_6' in model:
        list_magnitude_trans.append(6)

    if 'rot_6_' in model:
        list_magnitude_rot.append(6)
    elif 'rot_12' in model:
        list_magnitude_rot.append(12)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Rotation - Translation : second round

In [123]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_0' in model:
        list_magnitude_trans.append(0)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_6' in model:
        list_magnitude_trans.append(6)

    if 'rot_6_' in model:
        list_magnitude_rot.append(6)
    elif 'rot_12' in model:
        list_magnitude_rot.append(12)
    elif 'rot_18' in model:
        list_magnitude_rot.append(18)
    elif 'rot_24' in model:
        list_magnitude_rot.append(24)
    elif 'rot_30' in model:
        list_magnitude_rot.append(30)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Rotation THEN translation

In [42]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude_trans=[]
list_magnitude_rot=[]
for model in df['model'].tolist():
    if 'trans_1' in model:
        list_magnitude_trans.append(1)
    elif 'trans_3' in model:
        list_magnitude_trans.append(3)
    elif 'trans_5' in model:
        list_magnitude_trans.append(5)

    if 'rot_18' in model:
        list_magnitude_rot.append(18)
    elif 'rot_30' in model:
        list_magnitude_rot.append(30)


df['Rotation'] = list_magnitude_rot
df['Translation']=list_magnitude_trans
parameters=['Rotation', 'Translation']

Rotation only

In [142]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'rot_6_' in model:
        list_magnitude.append(6)
    elif 'rot_12' in model:
        list_magnitude.append(12)
    elif 'rot_18' in model:
        list_magnitude.append(18)
    elif 'rot_24' in model:
        list_magnitude.append(24)
    elif 'rot_30' in model:
        list_magnitude.append(30)
    elif 'rot_45' in model:
        list_magnitude.append(45)
    elif 'rot_60' in model:
        list_magnitude.append(60)
    elif 'rot_90' in model:
        list_magnitude.append(90)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Translation

In [156]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'translation_1_' in model:
        list_magnitude.append(1)
    elif 'translation_2' in model:
        list_magnitude.append(2)
    elif 'translation_3' in model:
        list_magnitude.append(3)
    elif 'translation_4' in model:
        list_magnitude.append(4)
    elif 'translation_5' in model:
        list_magnitude.append(5)
    elif 'translation_6' in model:
        list_magnitude.append(6)
    elif 'translation_7' in model:
        list_magnitude.append(7)
    elif 'translation_8' in model:
        list_magnitude.append(8)
    elif 'translation_9' in model:
        list_magnitude.append(9)
    elif 'translation_10' in model:
        list_magnitude.append(10)

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)
    elif 'p100' in model:
        list_proba.append(100) 

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

Trimdepth

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_mode=[]
list_proba=[]
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    elif '6mm' in model:
        list_magnitude.append(6)
    elif '8mm' in model:
        list_magnitude.append(8)

    if 'constant' in model:
        list_mode.append('constant')
    else:
        list_mode.append('per branch')

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Magnitude']=list_magnitude
df['Mode']=list_mode
df['Proba'] = list_proba
parameters=['Magnitude', 'Mode', 'Proba']

Trimextremities

In [31]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
for model in df['model'].tolist():
    if 'pepper50' in model:
        list_magnitude.append(50)
    elif 'pepper100' in model:
        list_magnitude.append(100)

    if 'p20' in model:
        list_proba.append(20) 
    elif 'p40' in model:
        list_proba.append(40)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
parameters=['Magnitude', 'Proba']

cutin

In [169]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '5_15' in model:
        list_magnitude.append('[05,15]')
    elif '10_30' in model:
        list_magnitude.append('[10,30]')
    elif '20_40' in model:
        list_magnitude.append('[20,40]')
    elif '30_50' in model:
        list_magnitude.append('[30,50]')
    elif '40_60' in model:
        list_magnitude.append('[40,60]')
    elif '50_70' in model:
        list_magnitude.append('[50,70]')
    elif '70_90' in model:
        list_magnitude.append('[70,90]')
    elif '20_70' in model:
        list_magnitude.append('[20,70]')
    elif '20_90' in model:
        list_magnitude.append('[20,90]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

cutout

In [181]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_proba=[]
list_bottom_proba=[]
for model in df['model'].tolist():
    if 'bottom50' in model:
        list_bottom_proba.append(50)
    elif 'bottom70' in model:
        list_bottom_proba.append(70)

    if '01_5' in model:
        list_magnitude.append('[01,5]')
    elif '5_15' in model:
        list_magnitude.append('[05,15]')
    elif '10_20' in model:
        list_magnitude.append('[10,20]')
    elif '15_25' in model:
        list_magnitude.append('[15,25]')
    elif '20_30' in model:
        list_magnitude.append('[20,30]')
    elif '30_40' in model:
        list_magnitude.append('[30,40]')
    elif '40_50' in model:
        list_magnitude.append('[40,50]')
    elif '50_60' in model:
        list_magnitude.append('[50,60]')
    elif '60_70' in model:
        list_magnitude.append('[60,70]')
    elif '70_80' in model:
        list_magnitude.append('[70,80]')
    elif '5_30' in model:
        list_magnitude.append('[05,30]')
    elif '5_40' in model:
        list_magnitude.append('[05,40]')

    if 'p40' in model:
        list_proba.append(40) 
    elif 'p80' in model:
        list_proba.append(80)

df['Proba'] = list_proba
df['Magnitude']=list_magnitude
df['Bottom_Proba']=list_bottom_proba
parameters=['Magnitude', 'Bottom_Proba', 'Proba']

In [7]:
# combinations
list_cutin = []
list_cutout = []
list_tr = []
list_rot = []

for model in df['model'].tolist():

    if 'cutin' in model:
        list_cutin.append('[20,90]')
    else:
        list_cutin.append(None)

    if 'cutout'in model:
        list_cutout.append('[5,30]')
    else:
        list_cutout.append(None)

    if 'max_translation_1' in model:
        list_tr.append(1)
    elif 'max_translation_3' in model:
        list_tr.append(3)
    elif 'max_translation_5' in model:
        list_tr.append(5)
    else:
        list_tr.append(None)

    if 'max_angle_12' in model:
        list_rot.append(12)
    elif 'max_angle_18' in model:
        list_rot.append(18)
    elif 'max_angle_30' in model:
        list_rot.append(30)
    else:
        list_rot.append(None)

df['cutin'] = list_cutin
df['cutout'] = list_cutout
df['translation'] = list_tr
df['rotation'] = list_rot

parameters = ['cutin', 'cutout', 'translation', 'rotation']

best config with trim

In [ ]:
# reshape the df so that keep p is a column
# all and bottom are also in a column
list_magnitude=[]
list_trimdepth = []
list_trimextremities = []
for model in df['model'].tolist():
    if '2mm' in model:
        list_magnitude.append(2)
    elif '4mm' in model:
        list_magnitude.append(4)
    else:
        list_magnitude.append(0)

    if 'trimdepth' in model:
        list_trimdepth.append('yes')
    else:
        list_trimdepth.append('no')

    if 'trimextremities' in model:
        list_trimextremities.append('yes')
    else:
        list_trimextremities.append('no')

df['Magnitude']=list_magnitude
df['Trimdepth']=list_trimdepth
df['Trimextremities'] = list_trimextremities
parameters=['Magnitude', 'Trimdepth', 'Trimextremities']

ValueError: Length of values (60) does not match length of index (75)

# Create tables

In [8]:
df_reformat = df.copy()
df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
df_reformat = df_reformat.drop(labels='model', axis='columns')
df_reformat['training_number'] = df_reformat.groupby(parameters+['task', 'region']).cumcount()
print('Check number of models for each config')
print(np.unique(df_reformat.groupby(parameters+['task', 'region']).cumcount(), return_counts=True))
## remove rows when more than 2 models
#df_reformat = df_reformat.drop(index=df_reformat.loc[df_reformat['training_number']>1].index.tolist(), axis='row')
parameters.append('training_number')

Check number of models for each config
(array([ 0., nan]), array([15, 29]))


In [9]:
#df_reformat = df.copy()
#df_reformat = df_reformat.drop_duplicates(['model', 'task']).reset_index(drop=True)
#df_reformat = df_reformat.drop(labels='model', axis='columns')
#df_reformat['training_number']=[k%n_models for k in range(len(df_reformat))]
#parameters.append('training_number')

In [11]:
# Pivoting the table
df_pivot = df_reformat.pivot(index=parameters, columns=['task', 'region'], values=['score', 'std', 'splits_score'])
df_pivot = df_pivot.dropna()

# Renaming columns
df_pivot.columns = [f'score_{col}' for col in df_pivot.columns]

# Resetting index to get a flat DataFrame
df_pivot = df_pivot.reset_index()

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("SC-sylv" in elem)]
df_pivot['score_Isomap_SC-sylv_left_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

isomap_score_cols = [elem for elem in df_pivot.columns if ("score_('score', 'Isomap" in elem) and ("Cing" in elem)]
df_pivot['score_Isomap_Cing_right_global'] = df_pivot[isomap_score_cols].astype(float).mean(axis=1)

In [12]:
df_scores = df_pivot[["score_('score', 'Left_OFC', 'SOr')", "score_('score', 'Right_FIP', 'FIP')", "score_('score', 'Right_PCS', 'Cing_2')",
                      "score_Isomap_SC-sylv_left_global", "score_Isomap_Cing_right_global"]].apply(pd.to_numeric).mul(100).round(1)
df_scores.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# get stds for Isomap
# SC-sylv
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_central_left_dim{k}', 'SC-sylv')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_sc_sylv = avg_per_split.apply(lambda row : np.std(row.tolist()))
# Cing
df_splits_score_isomap = df_pivot[[f"score_('splits_score', 'Isomap_cingulate_right_dim{k}', 'Cing_1')" for k in range(1,7)]]
avg_per_split = df_splits_score_isomap.apply(lambda row: np.mean(row.tolist(), axis=0), axis=1)
std_isomap_cing = avg_per_split.apply(lambda row : np.std(row.tolist()))

# get the std df
df_stds = df_pivot[["score_('std', 'Left_OFC', 'SOr')", "score_('std', 'Right_FIP', 'FIP')", "score_('std', 'Right_PCS', 'Cing_2')"]]
df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
df_stds['isomap_Cing_right']=std_isomap_cing
df_stds = df_stds.apply(pd.to_numeric).mul(100).round(1)
df_stds.columns=['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left', 'Isomap_Cing_right']

# combine in one df
formatted_df = df_scores.astype(str) + "(" + df_stds.astype(str) + ")"

formatted_df = pd.concat((df_pivot[parameters], formatted_df), axis=1)

/tmp/ipykernel_395982/1976764873.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_SC-sylv_left']=std_isomap_sc_sylv
/tmp/ipykernel_395982/1976764873.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stds['isomap_Cing_right']=std_isomap_cing


In [13]:
from scipy.stats import rankdata

# just use scores, no std
df_scores_only = pd.concat((df_pivot[parameters[:-1]], df_scores), axis=1)

# Compute mean metric per method
if len(parameters)==1: # when no augm
    df_mean = df_scores_only.mean().round(1)
    df_mean = df_mean.to_frame().transpose()
else:
    df_mean = df_scores_only.groupby(parameters[:-1]).mean().round(1)
# Rank each metric (lower values are better)
ranks = df_mean.apply(lambda x: rankdata(-x, method='average'), axis=0)

# Compute average rank
df_mean["Avg. Rank"] = ranks.mean(axis=1).round(1)

In [14]:
formatted_df

,cutin,cutout,translation,rotation,training_number,SOr,FIP,PCS,Isomap_SC-sylv_left,Isomap_Cing_right
0,"[20,90]",NaN,1,12,NaN,80.7(1.2),82.3(2.4),87.4(6.2),51.8(4.2),12.7(3.2)
1,"[20,90]","[5,30]",1,18,0.0,80.0(1.0),82.6(4.3),88.2(6.2),55.1(3.7),12.9(3.2)


In [15]:
df_mean

,,,,SOr,FIP,PCS,Isomap_SC-sylv_left,Isomap_Cing_right,Avg. Rank
cutin,cutout,translation,rotation,,,,,,
"[20,90]","[5,30]",1,18,80.0,82.6,88.2,55.1,12.9,1.0


# Save df

In [35]:
formatted_df.to_csv(save_dir+'.csv', index=False)

In [36]:
df_mean.to_csv(save_dir+'rank.csv', index=False)

## find best config in df scores

In [81]:
from scipy.stats import zscore

df_values = df_scores_only[['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left']]
df_zscores = df_values.apply(zscore)
formatted_df['average_zscore'] = df_zscores.mean(axis=1)

In [89]:
final_df = formatted_df.drop(['Isomap_Cing_right', 'training_number'], axis=1).sort_values('average_zscore', ascending=False).reset_index(drop=True)
final_df

,cutin,cutout,translation,rotation,SOr,FIP,PCS,Isomap_SC-sylv_left,average_zscore
0,"[20,90]",NaN,1.0,12.0,81.0(0.9),84.0(2.3),87.0(6.6),51.6(3.6),0.894548
1,"[20,90]","[5,30]",1.0,18.0,79.1(0.9),82.9(3.5),88.3(6.0),54.0(4.1),0.888747
2,"[20,90]",NaN,1.0,18.0,81.6(2.0),82.2(4.5),87.8(6.5),51.7(4.1),0.856186
3,"[20,90]","[5,30]",1.0,12.0,79.4(0.8),83.0(5.6),87.6(6.6),53.8(4.2),0.852994
4,"[20,90]","[5,30]",3.0,12.0,76.6(1.8),84.4(2.4),88.4(4.7),51.9(4.6),0.847381
5,"[20,90]",NaN,1.0,30.0,78.7(1.4),84.1(3.2),88.0(6.0),47.3(4.3),0.780650
6,"[20,90]",NaN,3.0,12.0,77.6(2.7),83.2(3.7),88.3(5.0),48.7(4.7),0.726869
7,"[20,90]",NaN,3.0,18.0,78.7(2.1),82.5(1.6),88.0(6.5),48.6(4.7),0.700633
8,NaN,"[5,30]",1.0,12.0,76.5(1.4),81.5(5.6),87.6(6.3),55.4(4.1),0.666452
9,"[20,90]","[5,30]",1.0,30.0,77.3(1.3),83.0(4.8),88.0(5.0),47.3(3.6),0.647344


In [126]:
## Assess the contribution of each augmentation
# average scores on each task per augm

# for each augm and parameter
augmentations = {'cutin': ['[20,90]'], 'cutout': ['[5,30]'], 'translation' : [1,3,5], 'rotation': [12,18,30]}
for augmentation, parameters in augmentations.items():
    print(augmentation)
    for parameter in parameters:
        print(parameter)
        df_augm=df_scores_only.loc[df_scores_only[augmentation]==parameter]
        if augmentation=='translation':
            df_augm
        df_no_augm=df_scores_only.loc[df_scores_only[augmentation].isna()]
        for task in ['SOr', 'FIP', 'PCS', 'Isomap_SC-sylv_left']:
            print(f'{task} : {df_augm[task].mean() - df_no_augm[task].mean()}')
        print('')

cutin
[20,90]
SOr : 7.9670087976539605
FIP : -0.8997067448680269
PCS : 1.205571847507315
Isomap_SC-sylv_left : 3.3231671554252316

cutout
[5,30]
SOr : 1.0330459770114828
FIP : -0.9686781609195378
PCS : 1.4617816091953983
Isomap_SC-sylv_left : 10.01120689655172

translation
1
SOr : 4.059999999999988
FIP : 5.496666666666684
PCS : 3.1133333333333297
Isomap_SC-sylv_left : 1.7633333333333212

3
SOr : 0.06769230769231172
FIP : 5.6899999999999835
PCS : 4.181538461538466
Isomap_SC-sylv_left : -6.522307692307692

5
SOr : -2.586666666666673
FIP : 3.416666666666657
PCS : 2.6666666666666714
Isomap_SC-sylv_left : -11.576666666666682

rotation
12
SOr : 0.9711111111111137
FIP : 2.928888888888892
PCS : 0.8911111111111154
Isomap_SC-sylv_left : -4.7333333333333485

18
SOr : 0.224444444444444
FIP : 3.0955555555555634
PCS : 2.1444444444444457
Isomap_SC-sylv_left : -5.7066666666666634

30
SOr : -1.3388888888889028
FIP : 2.06507936507937
PCS : 1.3634920634920604
Isomap_SC-sylv_left : -15.871428571428574



In [123]:
df_augm

,cutin,cutout,translation,rotation,SOr,FIP,PCS,Isomap_SC-sylv_left,Isomap_Cing_right,average_zscore
0,NaN,NaN,1.0,12.0,70.0,82.2,74.9,33.3,12.7,-0.972501
1,NaN,NaN,1.0,18.0,65.2,83.4,84.8,40.3,11.3,-0.216351
2,NaN,NaN,1.0,30.0,63.5,84.4,88.3,41.6,11.6,0.063604
3,NaN,NaN,3.0,12.0,63.1,83.3,83.5,30.2,11.2,-0.633149
4,NaN,NaN,3.0,18.0,60.5,84.1,86.1,25.6,11.4,-0.602964
5,NaN,NaN,3.0,30.0,59.8,84.7,84.9,20.2,12.0,-0.799984
6,NaN,NaN,5.0,12.0,64.7,82.8,78.4,29.8,9.1,-0.978016
7,NaN,NaN,5.0,18.0,60.2,84.1,77.6,29.1,9.5,-1.149107
8,NaN,NaN,5.0,30.0,58.4,84.5,84.3,15.7,10.4,-1.017069
9,NaN,"[5,30]",NaN,12.0,70.5,76.8,85.5,50.3,14.8,-0.171072


In [112]:
df_augm['SOr']

0     81.0(0.9)
1     79.1(0.9)
2     81.6(2.0)
3     79.4(0.8)
4     76.6(1.8)
5     78.7(1.4)
6     77.6(2.7)
7     78.7(2.1)
9     77.3(1.3)
10    75.6(2.0)
11    77.0(2.2)
13    77.6(2.8)
14    75.5(1.6)
15    72.6(1.4)
16    74.4(2.3)
17    75.0(2.7)
19    76.3(2.7)
24    73.0(2.4)
25    76.5(1.8)
27    75.0(2.2)
29    74.1(2.8)
30    73.8(2.8)
31    71.0(2.5)
32    71.1(1.3)
35    63.0(5.0)
36    71.7(2.9)
39    71.0(0.9)
40    73.2(2.5)
42    66.8(1.5)
51    64.7(1.6)
52    66.9(3.9)
Name: SOr, dtype: object